In [33]:
import json, pickle
import numpy as np
from sklearn.feature_extraction import text
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform

In [51]:
with open('results_test.json') as f:
    tokens_mini = json.load(f)
with open('stopwords.txt') as f:
    stopwords = set(f.read().split())

    
reducedDim = 10

In [52]:
def get_doc_iterator(doc_token_collection, by_sentence=False, indiv_tokens = False, stemmed=False,
                     stopwords=None, filter_fn = lambda x: True, get_key = False, skip_form_letters=False, 
                     condense_form_letters=False):
    form_letter_appearance=set()
    for item in doc_token_collection:
        k = item['isFormLetter']
        v = item['commentTextProc'] 
        names = item['complaintID']
        if skip_form_letters:
            if k[0:3]!="nfl":
                continue
        if condense_form_letters:
            if k[0:3]!="nfl": 
                if k[0:3] not in form_letter_appearance:
                    form_letter_appearance.add(k[0:3])
                else:
                    continue
        if filter_fn(v):
            idx = 2 if stemmed else 0
            if indiv_tokens:
                sentences = [[x[idx] for x in y if not stopwords or x[0] not in stopwords] for y in v]
            else:
                sentences = [' '.join([x[idx] for x in y]) for y in v]
            if by_sentence:
                for s in sentences:
                    yield s
            else:
                if get_key:
                    yield k
                else:
                    if indiv_tokens:
                        yield list(chain(*sentences))
                    else:
                        yield ' '.join(sentences)
                        

In [53]:
def get_doc_name_iterator(doc_token_collection, by_sentence=False, indiv_tokens = False, stemmed=False,
                     stopwords=None, filter_fn = lambda x: True, get_key = False, skip_form_letters=False, 
                     condense_form_letters=False):
    form_letter_appearance=set()
    for item in doc_token_collection:
        k = item['isFormLetter']
        v = item['commentTextProc'] 
        names = item['complaintID']
        if skip_form_letters:
            if k[0:3]!="nfl":
                continue
        if condense_form_letters:
            if k[0:3]!="nfl": 
                if k[0:3] not in form_letter_appearance:
                    form_letter_appearance.add(k[0:3])
                else:
                    continue
        if filter_fn(v):
            idx = 2 if stemmed else 0
            if indiv_tokens:
                sentences = [[x[idx] for x in y if not stopwords or x[0] not in stopwords] for y in v]
            else:
                sentences = [' '.join([x[idx] for x in y]) for y in v]
            if by_sentence:
                for s in sentences:
                    yield s
            else:
                if get_key:
                    yield k
                else:
                    if indiv_tokens:
                        yield list(chain(*sentences))
                    else:
                        yield (names,' '.join(sentences))
                        

In [54]:
#BAG OF WORDS IMPL:


In [78]:
sklearn_doc_iterator = get_doc_iterator(tokens_mini, stopwords=['xxxx', 'xx', '00'], indiv_tokens=False, stemmed=True, condense_form_letters=True)
count_vectorizer = text.CountVectorizer(stop_words=stopwords)
bow_matrix = count_vectorizer.fit_transform(sklearn_doc_iterator) #returns term-document matrix
words = np.array(count_vectorizer.get_feature_names())

In [79]:
print bow_matrix.shape
print words.shape
tsvd_bow = TruncatedSVD(reducedDim)
tsvd_bow.fit(bow_matrix)

(497, 3487)
(3487,)


TruncatedSVD(algorithm='randomized', n_components=10, n_iter=5,
       random_state=None, tol=0.0)

In [80]:
for i in range(reducedDim):
    row = tsvd_bow.components_[i,:]
    argsorted = np.argsort(row)[::-1][:15]
    scores = row[argsorted]
    top_words = words[argsorted]
    print ' , '.join(top_words)

xxxx , wa , thi , payment , call , xx , account , credit , 00 , loan , bank , receiv , time , told , report
wa , payment , thi , xx , credit , account , told , call , loan , 00 , time , receiv , pay , make , due
xx , credit , report , account , letter , compani , lender , card , ha , inform , date , remov , file , crc , disput
xx , payment , 00 , mortgag , month , ocwen , late , monthli , statement , due , loan , escrow , amount , date , properti
00 , credit , payment , report , statement , 15 , due , boa , amount , made , fee , late , balanc , paid , date
loan , mortgag , report , ocwen , credit , chase , thi , modif , file , payment , servic , ha , state , bureau , compani
thi , payment , bank , mortgag , account , complaint , america , statement , month , check , 00 , late , compani , fee , amount
account , loan , wa , servic , mortgag , close , ocwen , properti , escrow , check , contact , request , document , save , due
payment , account , becaus , chase , report , mortgag , ocwen

In [60]:
ref_docs = list(get_doc_name_iterator(tokens_mini, indiv_tokens=False, stemmed=False, condense_form_letters=True))


In [61]:
U = tsvd_bow.transform(bow_matrix)
V = tsvd_bow.components_

print U.shape
print V.shape

(497, 10)
(10, 4890)


In [62]:
for i, row in enumerate(U.T):
    argsort = np.argsort(row)[::-1]
    topic_word_sorted = np.argsort(V[i,:])[::-1]
    print i
    print 'Topic words: '
    print ' , '.join(words[topic_word_sorted][:20])
    print 'Example 1: '
    print ref_docs[argsort[0]]
    print 'Example 2: '
    print ref_docs[argsort[1]]
    print ' '

0
Topic words: 
xxxx , xx , credit , 00 , account , payment , bank , loan , told , mortgage , payments , time , received , letter , called , information , call , report , debt , back
Example 1: 
(u'1297691', u"from approximately xxxx xxxx - xxxx xxxx , i was appointed as an elder and treasurer of an xxxx ; xxxx xxxx upon appointment the leaders informally changed the name of the group from xxxx to xxxx the xxxx leaders were begrudgingly appointed by the owner of the property where meetings were held the owner , xxxx xxxx , transferred associated expenses for the building/property out of his name xxxx such expense was xxxx xxxx xxxx , which provided fuel for the facility without my knowledge or consent , this account was apparently placed into my personal name i had been informed that the bills would be sent to my personal address , as the treasurer of xxxx , due to the lack of a mailing address after several disagreements with xxxx xxxx , regarding xxxx and legal matters , i separated 

In [63]:
#TFIDF attempt:

In [64]:
sklearn_doc_iterator = get_doc_iterator(tokens_mini, indiv_tokens=False, stemmed=False, condense_form_letters=True)
#tfidf_vectorizer = text.TfidfVectorizer(stop_words=stopwords)
tfidf_vectorizer = text.TfidfVectorizer(min_df=2, stop_words = 'english',
strip_accents = 'unicode', lowercase=True, ngram_range=(1,2),
norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

tfidf_matrix = tfidf_vectorizer.fit_transform(sklearn_doc_iterator)
words_tfidf = np.array(tfidf_vectorizer.get_feature_names())

In [65]:
tsvd_tfidf = TruncatedSVD(reducedDim)
tsvd_tfidf.fit(tfidf_matrix)
V_tf = tsvd_tfidf.components_
U_tf = tsvd_tfidf.transform(tfidf_matrix)
print V_tf.shape
print U_tf.shape

(10, 5957)
(497, 10)


In [86]:
topic_words={}
for i in range(reducedDim):
    row = tsvd_tfidf.components_[i,:]
    argsorted = np.argsort(row)[::-1][:15]
    scores = row[argsorted]
    top_words = words_tfidf[argsorted]
    topic_words[i]=' , '.join(top_words)
for i in topic_words:
    print i
    print topic_words[i]

0
xxxx , xxxx xxxx , xx , credit , account , 00 , xx xx , bank , xx xxxx , payment , loan , report , mortgage , told , debt
1
xx , xx xx , xx xxxx , credit , xxxx xx , mortgage , account , payment , late , payments , card , bank , monthly , company , report
2
credit , report , credit report , account , debt , company , reporting , card , paid , information , sent , told , credit card , removed , letter
3
report , credit , credit report , xxxx xxxx , xx , reporting , debt , xxxx , equifax , experian , xx xx , xx xxxx , collection , transunion , dispute
4
account , bank , card , america , bank america , 00 , credit , balance , accounts , closed , credit card , fee , opened , checking , check
5
debt , number , phone , called , calls , calling , card , account , collection , agency , phone number , collect , said , collection agency , recovery
6
bank , identity theft , victim identity , identity , theft , certified , fraudulent , victim , america , bank america , sent , attached , account 

In [67]:
print tfidf_matrix.shape
print words_tfidf.shape
print tsvd_tfidf.components_.shape
print tsvd_tfidf.components_[0]

(497, 5957)
(5957,)
(10, 5957)
[ 0.09899865  0.00073097  0.00277033 ...,  0.00131856  0.00254517
  0.00192086]


In [68]:
for i, row in enumerate(U_tf.T):
    argsort = np.argsort(row)[::-1]
    topic_word_sorted = np.argsort(V_tf[i,:])[::-1]
    print i
    print 'Topic words: '
    print ' , '.join(words_tfidf[topic_word_sorted][:20])
    print 'Example 1: ' #for every topic, the highest ranking document.
    print ref_docs[argsort[0]]
    print 'Example 2: '
    print ref_docs[argsort[1]]
    print ' '

0
Topic words: 
xxxx , xxxx xxxx , xx , credit , account , 00 , xx xx , bank , xx xxxx , payment , loan , report , mortgage , told , debt , did , payments , received , time , called
Example 1: 
(u'1297691', u"from approximately xxxx xxxx - xxxx xxxx , i was appointed as an elder and treasurer of an xxxx ; xxxx xxxx upon appointment the leaders informally changed the name of the group from xxxx to xxxx the xxxx leaders were begrudgingly appointed by the owner of the property where meetings were held the owner , xxxx xxxx , transferred associated expenses for the building/property out of his name xxxx such expense was xxxx xxxx xxxx , which provided fuel for the facility without my knowledge or consent , this account was apparently placed into my personal name i had been informed that the bills would be sent to my personal address , as the treasurer of xxxx , due to the lack of a mailing address after several disagreements with xxxx xxxx , regarding xxxx and legal matters , i separated f

In [220]:
#Form letters filtered out:
#strategy 1: turn form letter into SINGLE data instance and place it - how to weigh it? 
#strategy 2: remove form letters altogether and return separately - but where are they located in proximity to rest?
#strategy 3: turn form letter into its own word, and use it with all instances.  loses similarity of form letter to rest...
#strategy 4: weaken form letter by a factor - logistic appearance, for example. 
#tried strategy 1 above. form letter weakened to single instance but is placed in space.


In [70]:
# #tsne_viz(mat=ww_ppmi[0], rownames=ww_ppmi[1])
# sklearn_doc_iterator = get_doc_iterator(tokens_mini, indiv_tokens=False, stemmed=False)
# tfidf_fortsne_vectorizer = text.TfidfVectorizer(min_df=2, stop_words = 'english',
# strip_accents = 'unicode', lowercase=True, ngram_range=(1,2),
# norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
# X = tfidf_fortsne_vectorizer.fit_transform(sklearn_doc_iterator)
# print X.shape
# D = -(X * X.T).todense() # Distance matrix: dot product between tfidf vectors
# print D.shape

#http://karpathy.github.io/2014/07/02/visualizing-top-tweeps-with-t-sne-in-Javascript/


In [93]:
def tsne_viz(
        mat,
        rownames,
        colors=None,
        output_filename=None,
        figheight=40,
        figwidth=50):     
    """2d plot of `mat` using t-SNE, with the points labeled by `rownames`, 
    aligned with `colors` (defaults to all black).
    
    Parameters
    ----------    
    mat : 2d np.array
        The matrix to visualize.
        
    rownames : list of str
        Names of the points to visualize.
                
    colors : list of colornames or None (default: None)
        Optional list of colors for rownames. The color names just need to 
        be interpretable by matplotlib. If they are supplied, they need to 
        have the same length as rownames, or indices if that is not None. 
        If `colors=None`, then all the words are displayed in black.
      
    output_filename : str (default: None)
        If not None, then the output image is written to this location. The 
        filename suffix determines the image type. If None, then 
        `plt.plot()` is called, with the behavior determined by the 
        environment.
        
    figheight : int (default: 40)
        Height in display units of the output.
            
    figwidth : int (default: 50)
        Width in display units of the output.
        
    """
    indices = list(range(len(rownames)))
    # Colors:
    if not colors:
        colors = ['black' for i in indices]    
    # Recommended reduction via PCA or similar:
    n_components = 50 if mat.shape[1] >= 50 else mat.shape[1]
    dimreduce = PCA(n_components=n_components)
    mat = dimreduce.fit_transform(mat)
    
    # t-SNE:
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)    
    tsnemat = tsne.fit_transform(mat) 
   
    # Plot values:
    vocab = np.array(rownames)[indices]
    xvals = tsnemat[indices, 0] 
    yvals = tsnemat[indices, 1]
    # Plotting:
    fig, ax = plt.subplots(nrows=1, ncols=1)
#     fig.set_figheight(40)
#     fig.set_figwidth(50)
    ax.plot(xvals, yvals, marker='', linestyle='')
    # Text labels:
    for word, x, y, color in zip(vocab, xvals, yvals, colors):
        ax.annotate(word, (x, y), fontsize=8, color=color)
    # Output:
    if output_filename:
        plt.savefig(output_filename, bbox_inches='tight')
    else:
        plt.show()
        
    

def plot_embedding(X, doc_to_topic_matrix, title=None):
    #try tsne on clustering
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    labels =  np.argmax(doc_to_topic_matrix, axis=1)
    for i in range(X.shape[0]):
        
        plt.text(X[i, 0], X[i, 1], labels[i],
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [94]:
print ref_docs[0]

(u'1290516', u'xxxx has claimed i owe them { $ 27 00 } for xxxx years despite the proof of payment i sent them : canceled check and their ownpaid invoice for { $ 27 00 } ! they continue to insist i owe them and collection agencies are after me how can i stop this harassment for a bill i already paid four years ago ?')


In [96]:
# tsne_viz(tfidf_matrix.toarray(), np.argmax(U_tf, axis=1))

# tsne_viz(U_tf, np.argmax(U_tf, axis=1)) # U_tf is already reduced. 

rownames = np.argmax(U_tf, axis=1)#zip(np.argmax(U_tf, axis=1),[item[0] for item in ref_docs])
dists = pdist(tfidf_matrix.toarray(),'cosine') # or U_tf toarray? no
tsne_viz(squareform(dists), rownames)

# dist_matrix=(np.dot(tfidf_matrix.toarray(), tfidf_matrix.toarray().T))
# tsne_viz(dist_matrix, np.argmax(U_tf, axis=1))

# dist_U_matrix=(np.dot(U_tf, U_tf.T))  #so I don't think this makes sense, to use affiliations towards topics. we should use raw. 
# tsne_viz(dist_U_matrix, np.argmax(U_tf, axis=1))

In [ ]:
###NOT REAL CODE PAST THIS POINT! MY ATTEMPTS 

In [77]:

model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne=model.fit_transform(U) 
D_tsne = -(np.dot(X_tsne, X_tsne.T))#.todense() 
plot_embedding(D_tsne, U,
               "t-SNE embedding of the the comments")

plt.show()


In [131]:

model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne=model.fit_transform(bow_matrix) 
#D_tsne = -(np.dot(X_tsne, X_tsne.T))#.todense()
plot_embedding(X_tsne, U,
               "t-SNE embedding of the the comments")

plt.show()


In [155]:
print tfidf_matrix.shape #TFIDF matrix is not reduced, U is the reduced version!
print U_tf.shape
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne=model.fit_transform(U_tf) 
D_tsne = -(np.dot(X_tsne, X_tsne.T))
plot_embedding(D_tsne, U_tf,
               "t-SNE embedding of the the comments")

plt.show()


(236, 1954)
(236, 5)


In [156]:
print tfidf_matrix.shape #TFIDF matrix is not reduced, U is the reduced version!
print U_tf.shape
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne=model.fit_transform(tfidf_matrix) 
#D_tsne = -(np.dot(X_tsne, X_tsne.T))
plot_embedding(X_tsne, U_tf,
               "t-SNE embedding of the the comments")

plt.show()


(236, 1954)
(236, 5)


In [ ]:
#most common words should have very little influence? tfidf scores should be evry low... figure out how this is processed?

#check out gensim tut2 available transformations, try everything there. 

In [43]:
#now, filter out all form letters before advancing into clustering, and present form letters as own clusters WITH similarity to another cluster. 

In [ ]:
#continue per 229project, try larger dataset, with NER tags, with opinion tags, with negation tags... what if each doc was average of all of its words' word2vec?